# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [17]:
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
response = requests.get(url)
data = response.json()

In [18]:
df = pd.DataFrame(data['items'])

frequencia_status = df['status'].value_counts()
primeiraFrequencia = frequencia_status.index[0]
segundaFrequencia = frequencia_status.index[1]
terceiraFrequencia = frequencia_status.index[2]

print(f'Os status mais frequentes são: \n1º lugar: {primeiraFrequencia} com {frequencia_status[0]}\n2º Lugar {segundaFrequencia} com {frequencia_status[1]} \n3º lugar {terceiraFrequencia} com {frequencia_status[2]}')


Os status mais frequentes são: 
1º lugar: Encaminhado à PGFN com manifestação técnica favorável com 34
2º Lugar Deferido com 14 
3º lugar Arquivado a pedido com 3


In [11]:
anoStatus = df['data_status'].str[-4:]
anoStatus

0     2013
1     2011
2     2009
3     2010
4     2011
5     2011
6     2016
7     2012
8     2009
9     2012
10    2009
11    2006
12    2002
13    2014
14    2010
15    2016
16    2008
17    2016
18    2009
19    2009
20    2002
21    2012
22    2011
23    2011
24    2009
25    2012
26    2016
27    2011
28    2006
29    2009
30    2012
31    2012
32    2012
33    2009
34    2014
35    2013
36    2012
37    2002
38    2013
39    2010
40    2008
41    2012
42    2009
43    2013
44    2014
45    2014
46    2012
47    2012
48    2002
49    2013
50    2013
51    2013
Name: data_status, dtype: object

In [5]:
frequencia_anos = anoStatus.value_counts()
frequencia_anos

2012    11
2009     9
2013     7
2011     6
2016     4
2002     4
2014     4
2010     3
2006     2
2008     2
Name: data_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [48]:
#1) Seu código aqui 
#Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado
# ('Estado'ou Município), e que devolve os dados da consulta no formato DataFrame.
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

def consultar_dados_uf_tipo(uf, tipo):
    if tipo not in ['Estado', 'Município']:
        raise ValueError("O tipo de interessado deve ser 'Estado' ou 'Município'")

    # Configurar o driver do Selenium (certifique-se de ter o ChromeDriver instalado)
    driver = webdriver.Chrome()

    # Abrir a página da API
    driver.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl')

    # Selecionar UF
    uf_select = driver.find_element(By.XPATH, "//select[@name='uf']")
    uf_select.click()
    uf_option = uf_select.find_element(By.XPATH, f"//option[text()='{uf}']")
    uf_option.click()

    # Selecionar Tipo de Interessado
    tipo_select = driver.find_element(By.XPATH, "//select[@name='tipo_interessado']")
    tipo_select.click()
    tipo_option = tipo_select.find_element(By.XPATH, f"//option[text()='{tipo}']")
    tipo_option.click()

    # Enviar o formulário
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit']")
    submit_button.click()

    # Extrair dados da página
    data = []
    for row in driver.find_elements(By.XPATH, "//table[@id='results']//tr"):
        data.append([cell.text for cell in row.find_elements(By.TAG_NAME, "td")])

    # Fechar o navegador
    driver.quit()

    if len(data) > 1:
        return pd.DataFrame(data[1:], columns=data[0])
    else:
        raise ValueError("Não foram encontrados dados para a consulta.")

# Exemplo de uso da função:
uf = 'MG'
tipo = 'Estado'
df = consultar_dados_uf_tipo(uf, tipo)
print(df)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=118.0.5993.89)
Stacktrace:
	GetHandleVerifier [0x00007FF76B638EF2+54786]
	(No symbol) [0x00007FF76B5A5612]
	(No symbol) [0x00007FF76B45A64B]
	(No symbol) [0x00007FF76B43A9F5]
	(No symbol) [0x00007FF76B4C0887]
	(No symbol) [0x00007FF76B4D43CF]
	(No symbol) [0x00007FF76B4BBC43]
	(No symbol) [0x00007FF76B490941]
	(No symbol) [0x00007FF76B491B84]
	GetHandleVerifier [0x00007FF76B987F52+3524194]
	GetHandleVerifier [0x00007FF76B9DD800+3874576]
	GetHandleVerifier [0x00007FF76B9D5D7F+3843215]
	GetHandleVerifier [0x00007FF76B6D5086+694166]
	(No symbol) [0x00007FF76B5B0A88]
	(No symbol) [0x00007FF76B5ACA94]
	(No symbol) [0x00007FF76B5ACBC2]
	(No symbol) [0x00007FF76B59CC83]
	BaseThreadInitThunk [0x00007FFA5E9F7344+20]
	RtlUserThreadStart [0x00007FFA5FDA26B1+33]


In [54]:
#Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 
#'Arquivado por decurso de prazo' estão registradas?


url2 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
r = requests.get(url2)
data = r.json()
pd.DataFrame(data['items'])

data = {'status': ['Arquivado por decurso de prazo']}
df = pd.DataFrame(data)
contagem = df['status'].value_counts()
contagem

Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [81]:
# 3) Seu código aqui
# Qual é o município da Bahia com mais solicitações deferidas?

url3 = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio'
r2 = requests.get(url3)
data2 = r2.json()
df2 = pd.DataFrame(data2['items'])

contagem_status = df2['status'].value_counts()
deferido_df = df2[df2['status'] == 'Deferido']
contagem_municipio_deferido = deferido_df['interessado'].value_counts()
municipio_mais_deferido = contagem_municipio_deferido.idxmax()
quantidade_mais_deferido = contagem_municipio_deferido.max()
print(f"O município com mais status deferidos é {municipio_mais_deferido} com um total de {quantidade_mais_deferido} deferimentos.")

O município com mais status deferidos é Luís Eduardo Magalhães com um total de 16 deferimentos.


In [82]:
# 4) Seu código aqui 
#Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
solicitacoes_estado_bahia = df2[df2['interessado'] == 'Estado']
solicitacoes_estado_bahia.to_csv('solicitacoes_estado_bahia.csv', index=False)